# Tutorial: Leakage scheme implementation

## Author: Leo Werneck

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

1. [Step 1](#initialize_nrpy): Initialize NRPy+/Python modules
1. [Step 2](#declare_c_params): Declare C parameters
1. [Step 3](#fermi_dirac_integrals): Fermi-Dirac integrals
1. [Step 4](#free_emission_cooling_rates): Free neutrino emission and cooling rates
    1. [Step 4.a](#beta_processes): Electron and positron capture ($\beta$-processes)
        1. [Step 4.a.i](#degeneracy_parameters): Compute all degeneracy parameters
        1. [Step 4.a.ii](#energy_moments): Compute energy-moments
        1. [Step 4.a.iii](#beta_blocking_factors): Blocking factors
        1. [Step 4.a.iv](#beta_emission_rates): Free neutrino emission rates
    1. [Step 4.b](#pair_annihilation): Electron-positron pair annihilation
    1. [Step 4.c](#plasmon_decay): Transverse plasmon decay
    1. [Step 4.d](#bremsstrahlung): Nucleon-nucleon Bremsstrahlung
    1. [Step 4.e](#free_cooling_rates): Free neutrino cooling rates
    1. [Step 4.f](#total_rates): Total emission and cooling rates for free neutrinos
    1. [Step 4.g](#c_function_generation): C function generation
1. [Step 5](#writing_leakage_hh): Writing the C header file `Leakage.hh`
1. [Step 6](#c_code_generation): C code generation

<a id='initialize_nrpy'></a>

# Step 1: Initialize NRPy+/Python modules \[Back to [Top](#toc)\]
$$\label{initialize_nrpy}$$

In [1]:
import os,sys,shutil
import sympy as sp
import astropy.constants as ct
import astropy.units as units
sys.path.append("nrpy_core")
import outputC as outC
import NRPy_param_funcs as par
import cmdline_helper as cmd

Ccodesdir = os.path.join("standalone","NRPyLeakage")
shutil.rmtree(Ccodesdir,ignore_errors=True)
cmd.mkdir(Ccodesdir)

function_prototypes = []

<a id='declare_c_params'></a>

# Step 2: Declare C parameters \[Back to [Top](#toc)\]
$$\label{declare_c_params}$$

We start by implementing a series of constants that will be used by our C code. These appear in the various equations in appendices A and B of [Ruffert *et al.* (1996)](https://adsabs.harvard.edu/pdf/1996A%26A...311..532R). For convenience, we use the [Astropy](https://www.astropy.org/) package to fetch the values of physical constants and for unit conversion.

In [2]:
# C parameters
namespace  = "LEAKAGENAMESPACE"
thismodule = "Leakage"
Q_npmass   = par.Cparameters("REAL",thismodule,namespace+"Q_npmass"                 ,"1.2935")
gamma_0    = par.Cparameters("REAL",thismodule,namespace+"gamma_0"                  ,"5.565e-2")
c_light    = par.Cparameters("REAL",thismodule,namespace+"c_light"                  ,"2.997924580000000e+10")
N_A        = par.Cparameters("REAL",thismodule,namespace+"N_A"                      ,"6.022140760000000e+23")
sigma_0    = par.Cparameters("REAL",thismodule,namespace+"sigma_0"                  ,"1.76e-44")
alpha      = par.Cparameters("REAL",thismodule,namespace+"alpha"                    ,"1.25")
alpha_fs   = par.Cparameters("REAL",thismodule,namespace+"alpha_fs"                 ,"7.297352569300000e-03")
C_A        = par.Cparameters("REAL",thismodule,namespace+"C_A"                      ,"0.5")
sinthw2    = par.Cparameters("REAL",thismodule,namespace+"sinthw2"                  ,"0.23")
MeV_to_erg = par.Cparameters("REAL",thismodule,namespace+"MeV_to_erg"               ,"1.602176634000000e-06")
amu        = par.Cparameters("REAL",thismodule,namespace+"amu"                      ,"1.660539066600000e-24")
hc3        = par.Cparameters("REAL",thismodule,namespace+"hc3"                      ,"1.905895198207216e-30")
m_e_c2     = par.Cparameters("REAL",thismodule,namespace+"m_e_c2"                   ,"5.109989499961642e-01")
Brems_C1   = par.Cparameters("REAL",thismodule,namespace+"Brems_C1"                 ,"2.9988e7")
Brems_C2   = par.Cparameters("REAL",thismodule,namespace+"Brems_C2"                 ,"6.5428e7")
Brems_zeta = par.Cparameters("REAL",thismodule,namespace+"Brems_zeta"               ,"0.5")
eta_nue_0  = par.Cparameters("REAL",thismodule,namespace+"eta_nue_0"                ,"0.0")
eta_anue_0 = par.Cparameters("REAL",thismodule,namespace+"eta_anue_0"               ,"0.0")
Dgeomtocgs = par.Cparameters("REAL",thismodule,namespace+"units_geom_to_cgs_density","6.175828479261933e+17")
M_PI       = par.Cparameters("REAL","math","M_PI","3.141592653589793")

<a id='fermi_dirac_integrals'></a>

# Step 3: Fermi-Dirac integrals \[Back to [Top](#toc)\]
$$\label{fermi_dirac_integrals}$$

Below we implement a function to evaluate Fermi-Dirac integrals

$$
F_{N}(z) = \int_{0}^{\infty}dx\frac{x^{N}}{e^{z-x}+1}\; ,
$$

according to the approximations in [Takahashi, Eid, and Hillebrandt (1978)](https://adsabs.harvard.edu/pdf/1978A%26A....67..185T).

In [3]:
# For N=0, we have an analytic expression
x,z  = sp.symbols('x z',real=True)
one  = sp.sympify(1)
expz = sp.exp(z)
F_0  = sp.simplify(sp.integrate(1.0/(sp.exp(x-z)+1),(x,0,sp.oo)))

# N = 0 case
F_zlarge = [F_0]
F_zsmall = [F_0]

# N=1 case
# z > 1e-3
F_zlarge.append((sp.Rational(1,2)*z**2 + sp.sympify(1.6449))/(one+sp.exp(-sp.sympify(1.6855)*z)))
# z < 1e-3
F_zsmall.append(expz/( one + sp.sympify(0.2159)*sp.exp(sp.sympify(0.8857)*z) ))

# N=2 case
# z > 1e-3
F_zlarge.append((sp.Rational(1,3)*z**3 + sp.sympify(3.2899)*z)/(one - sp.exp(-sp.sympify(1.8246)*z)))
# z < 1e-3
F_zsmall.append(sp.sympify(2)*expz/( one + sp.sympify(0.1092)*sp.exp(sp.sympify(0.8908)*z) ))

# N=3 case
# z > 1e-3
F_zlarge.append((sp.Rational(1,4)*z**4 + sp.sympify(4.9348)*z**2 + sp.sympify(11.3644))/(one+sp.exp(-sp.sympify(1.9039)*z)))
# z < 1e-3
F_zsmall.append(sp.sympify(6)*expz/( one + sp.sympify(0.0559)*sp.exp(sp.sympify(0.9069)*z) ))

# N=4 case
# z > 1e-3
F_zlarge.append((sp.Rational(1,5)*z**5 + sp.sympify(6.5797)*z**3+sp.sympify(45.4576)*z)/(one-sp.exp(-sp.sympify(1.9484)*z)))
# z < 1e-3
F_zsmall.append(sp.sympify(24)*expz/( one + sp.sympify(0.0287)*sp.exp(sp.sympify(0.9257)*z) ))

# N=5 case
# z > 1e-3
F_zlarge.append((sp.Rational(1,6)*z**6 + sp.sympify(8.2247)*z**4 + sp.sympify(113.6439)*z**2 + sp.sympify(236.5323))/(one+sp.exp(-sp.sympify(1.9727)*z)))
# z < 1e-3
F_zsmall.append(sp.sympify(120)*expz/( one + sp.sympify(0.0147)*sp.exp(sp.sympify(0.9431)*z)))

def Cfunc_Leakage_Fermi_Dirac_Integrals():
    desc = """(c) Leo Werneck
Compute Fermi-Dirac integrals according to the approximations
in Takahashi, Eid, and Hillebrandt (1978)
https://adsabs.harvard.edu/pdf/1978A%26A....67..185T"""
    includes = ["Basic_defines.hh"]
    c_type = "extern \"C\"\nREAL"
    name   = "Leakage_Fermi_Dirac_integrals"
    params = "const int k, const REAL z"
    outCparams = "outCverbose=False,includebraces=False"
    prefunc = ""
    zlarge_body = ""
    zsmall_body = ""
    for k in range(6):
        zlarge_body += "    case("+str(k)+"):\n"
        zlarge_body += "      "+outC.outputC(F_zlarge[k],"Fermi_Dirac_integral","returnstring",params=outCparams)
        zsmall_body += "    case("+str(k)+"):\n"
        zsmall_body += "      "+outC.outputC(F_zsmall[k],"Fermi_Dirac_integral","returnstring",params=outCparams)
        if k < 5:
            zlarge_body += "      break;\n"
            zsmall_body += "      break;\n"
        else:
            zlarge_body += "      break;"
            zsmall_body += "      break;"
    body   = """
  REAL Fermi_Dirac_integral = 0.0;
  if(z>1e-3) {
    switch(k) {
"""+zlarge_body+r"""
    default:
      fprintf(stderr,"Unsuported value of k: %d\n",k);
      exit(1);
    }
  }
  else {
    switch(k) {
"""+zsmall_body+r"""
    default:
      fprintf(stderr,"Unsuported value of k: %d\n",k);
      exit(1);
    }
  }
  
  return Fermi_Dirac_integral;
"""
    loopopts = ""
    outC.outCfunction(os.path.join(Ccodesdir,name+".cc"),
                      includes=includes,desc=desc,prefunc=prefunc,c_type=c_type,name=name,
                      params=params,body=body,loopopts=loopopts,enableCparameters=False)

    global function_prototypes
    function_prototypes.append(c_type+" "+name+"("+params+");\n")
    function_prototypes = outC.superfast_uniq(function_prototypes)

def Fermi_Dirac_validation(k,z):
    if k > 5:
        print("Unsupported value of k")
        sys.exit(1)  
    if z > 1e-3:
        F = sp.lambdify(sp.symbols('z',real=True),F_zlarge[k])
    else:
        F = sp.lambdify(sp.symbols('z',real=True),F_zsmall[k])
    return F(z)

import fdint
for k in range(6):
    print("Order %d: %e"%(k,(1.0-fdint.fdk(k,1e-5)/Fermi_Dirac_validation(k,1e-5))))
    print("Order %d: %e"%(k,(1.0-fdint.fdk(k,1e-1)/Fermi_Dirac_validation(k,1e-1))))

Order 0: 0.000000e+00
Order 0: 1.110223e-16
Order 1: -3.766628e-05
Order 1: -1.662731e-05
Order 2: 8.862389e-06
Order 2: -5.354045e-05
Order 3: 2.803556e-05
Order 3: 2.572759e-06
Order 4: -1.960806e-05
Order 4: -2.903726e-05
Order 5: -3.869268e-05
Order 5: -3.966739e-07


<a id='free_emission_cooling_rates'></a>

# Step 4: Free neutrino emission and cooling rates \[Back to [Top](#toc)\]
$$\label{free_emission_cooling_rates}$$

<a id='beta_processes'></a>

## Step 4.a: Electron and positron capture ($\beta$-processes) \[Back to [Top](#toc)\]
$$\label{beta_processes}$$

For electrons captured by protons, we use (cf. Eqs. B1 and B2 in [Ruffert *et al.* (1996)](https://adsabs.harvard.edu/pdf/1996A%26A...311..532R))

$$
\newcommand{\mue}{\mu_{\rm e}}
\newcommand{\mun}{\mu_{\rm n}}
\newcommand{\mup}{\mu_{\rm p}}
\newcommand{\muhat}{\hat{\mu}}
\newcommand{\me}{m_{\rm e}}
\newcommand{\nb}{n_{\rm b}}
\newcommand{\ee}{e^{-}}
\newcommand{\ae}{e^{+}}
\newcommand{\nue}{\nu_{\rm e}}
\newcommand{\anue}{\bar{\nu}_{\rm e}}
\newcommand{\nui}{\nu_{\rm i}}
\newcommand{\anui}{\bar{\nu}_{\rm i}}
\newcommand{\nux}{\nu_{\rm x}}
\newcommand{\anux}{\bar{\nu}_{\rm x}}
\newcommand{\rhob}{\rho_{\rm b}}
\newcommand{\ye}{Y_{\rm e}}
\newcommand{\QQ}{\mathcal{Q}}
\newcommand{\RR}{\mathcal{R}}
\newcommand{\etae}{\eta_{\rm e}}
\newcommand{\etap}{\eta_{\rm p}}
\newcommand{\etan}{\eta_{\rm n}}
\newcommand{\etanue}{\eta_{\nue}}
\newcommand{\etaanue}{\eta_{\anue}}
\newcommand{\etanux}{\eta_{\nux}}
\newcommand{\etanui}{\eta_{\nui}}
\newcommand{\dif}{\mathrm{d}}
\newcommand{\rate}[3]{#1_{#3}^{#2}}
\newcommand{\etahat}{\hat{\eta}}
\begin{align}
  \rate{\RR}{\nue}{\rm ec} &= \frac{1+3\alpha^{2}}{8}\beta\,\rhob Y_{\rm pn}\varepsilon_{4}(+\etae)\langle1-f_{\nue}\rangle_{\rm ec}\;,\\
  %%%%
  \rate{\RR}{\anue}{\rm pc} &= \frac{1+3\alpha^{2}}{8}\beta\,\rhob Y_{\rm np}\varepsilon_{4}(-\etae)\langle1-f_{\anue}\rangle_{\rm pc}\;,
\end{align}
$$

where we have the blocking factors (cf. Eqs. B3 and B4 in [Ruffert *et al.* (1996)](https://adsabs.harvard.edu/pdf/1996A%26A...311..532R))
$$
\begin{align}
\langle1-f_{\nue}\rangle_{\rm ec} &= \left\{1 + \exp\left[\eta_{\nue}-\frac{F_{5}(\eta_{\rm e})}{F_{4}(\eta_{\rm e})}\right]\right\}^{-1}\; ,\\
\langle1-f_{\nue}\rangle_{\rm pc} &= \left\{1 + \exp\left[\eta_{\anue}-\frac{F_{5}(-\eta_{\rm e})}{F_{4}(-\eta_{\rm e})}\right]\right\}^{-1}\; ,
\end{align}
$$

where $\etae = \mue/T$ is the electron degeneracy parameter, $\mue$ is the electron chemical potential, and $T$ is the temperature. We have also defined the constant

$$
\beta = \frac{\sigma_{0}c}{(\me c^{2})^{2}}N_{A}\;,
$$

where $\me$ is the electron mass, $c$ is the speed of light, $N_{A}$ is Avogadro's constant, and $\sigma_{0}=1.76\times10^{-44}\,{\rm cm^{2}}$; the energy moments

$$
\varepsilon_{N}(\pm\etae) = \frac{8\pi}{(hc)^{3}}T^{N+1}F_{N}(\pm\etae)\;,
$$

where $h$ is Planck's constant; and the Fermi-Dirac integrals

$$
F_{k}(\eta) = \int_{0}^{\infty}\dif x\frac{x^{k}}{e^{x-\eta}+1}\;.
$$

In the blocking factor, the electron neutrino and antineutrino degeneracy parameters are computed using

$$
\begin{align}
\eta_{\nue} &= \eta_{\nue}^{\rm ceq} \left[1 - \exp(-\tau_{\nue})\right] + \eta_{\nue}^{0}\exp(-\tau_{\nue})\; ,\\
\eta_{\anue} &= \eta_{\anue}^{\rm ceq} \left[1 - \exp(-\tau_{\anue})\right] + \eta_{\anue}^{0}\exp(-\tau_{\anue})\; ,
\end{align}
$$

where $\eta_{\nue}^{0} = 0 = \eta_{\anue}^{0}$, $\tau_{\nue}$ and $\tau_{\anue}$ are the optical depths of the electron neutrino and antineutrino, respectively, and

$$
\eta_{\nue}^{\rm ceq} =  - \eta_{\anue}^{\rm ceq} = \eta_{\rm e} + \eta_{\rm p} - \eta_{\rm n} - Q/T\; ,
$$

where $\eta_{i}=\mu_{i}/T$, with $i={\rm e,n,p}$, and $Q=1.2935$ MeV is the rest-mass-energy difference between a neutron and a proton. Finally, the number fractions $Y_{\rm np}$ and $Y_{\rm pn}$ are defined as

$$
\begin{align}
Y_{\rm pn} &=
\left\{
\begin{array}{c}
\frac{2\ye-1}{1-e^{\etahat}}\;, &\text{if }\ye<0.5\;,\\
1-\ye\;,&\text{otherwise}\;,
\end{array}
\right.\\
%%%%%%%%%%
Y_{\rm np} &=
\left\{
\begin{array}{c}
1-\ye\;, &\text{if }\ye<0.5\;,\\
e^{\etahat}Y_{\rm pn}\;,&\text{otherwise}\;.
\end{array}
\right.
\end{align}
$$

We declare $Y_{\rm np}$ and $Y_{\rm pn}$ as SymPy symbols below, computing them "by hand" in the C code before they are needed.

<a id='degeneracy_parameters'></a>

### Step 4.a.i: Compute all degeneracy parameters \[Back to [Top](#toc)\]
$$\label{degeneracy_parameters}$$

We start by computing the degeneracy parameters

$$
\eta_{i} = \mu_{i}/T\;,
$$

for $i=e,n,p$.

In [4]:
# Step 4: Free neutrino emission and cooling rates
# Step 4.a: Electron and positron capture (beta processes)
# Step 4.a.i: Degeneracy parameters
# Step 4.a.i.A: Declare SymPy symbols required to compute degeneracy parameters
zero      = sp.sympify(0)
mu_e, mu_n, mu_p, muhat = sp.symbols("mu_e mu_n mu_p muhat",real=True)
Y_np, Y_pn, T, tau_nue, tau_anue = sp.symbols("Y_np Y_pn T tau_nue tau_anue",real=True)
eta_nue_0    = sp.symbols(namespace+"eta_nue_0",real=True)
eta_anue_0   = sp.symbols(namespace+"eta_anue_0",real=True)

# Step 4.a.i.B: Electron degeneracy parameter
eta_e        = mu_e / T

# Step 4.a.i.C: Neutron degeneracy parameter
eta_n        = mu_n / T

# Step 4.a.i.D: Proton degeneracy parameter
eta_p        = mu_p / T

# Step 4.a.i.E: Neutron-proton difference degeneracy parameter
eta_hat      = muhat / T

Now the neutrino degeneracy parameters:

$$
\begin{align}
\eta_{\nue} &= \eta_{\nue}^{\rm ceq} \left[1 - \exp(-\tau_{\nue})\right] + \eta_{\nue}^{0}\exp(-\tau_{\nue})\; ,\\
\eta_{\anue} &= \eta_{\anue}^{\rm ceq} \left[1 - \exp(-\tau_{\anue})\right] + \eta_{\anue}^{0}\exp(-\tau_{\anue})\; ,\\
\eta_{\nux} &= \eta_{\anux} = 0\;,
\end{align}
$$

where

$$
\eta_{\nue}^{\rm ceq} =  - \eta_{\anue}^{\rm ceq} = \eta_{\rm e} + \eta_{\rm p} - \eta_{\rm n} - Q/T\; .
$$

In [5]:
# Step 4.a.i.F: Electron neutrino degeneracy parameter (in equilibrium)
eta_nue_ceq = eta_e + eta_p - eta_n - Q_npmass/T

# Step 4.a.i.G: Electron antineutrino degeneracy parameter (in equilibrium)
eta_anue_ceq = -eta_nue_ceq

# Step 4.a.i.H: Heavy lepton neutrinos/antineutrinos degeneracy parameter
eta_nux = zero

# Step 4.a.i.I: Electron neutrino degeneracy parameter
eta_nue = eta_nue_ceq*(one-sp.exp(-tau_nue)) + eta_nue_0*sp.exp(-tau_nue)

# Step 4.a.i.J: Electron antineutrino degeneracy parameter
eta_anue = eta_anue_ceq*(one-sp.exp(-tau_anue)) + eta_anue_0*sp.exp(-tau_anue)

<a id='energy_moments'></a>

### Step 4.a.ii: Compute energy-moments \[Back to [Top](#toc)\]
$$\label{energy_moments}$$

We now compute the needed energy moments, $\varepsilon_{4}(+\etae)$ and $\varepsilon_{4}(-\etae)$, using

$$
\varepsilon_{N}(\pm\etae) = \frac{8\pi}{(hc)^{3}}T^{N+1}F_{N}(\pm\etae)\;.
$$

In [6]:
# Step 4.a.ii: Compute the electron energy moments
# Step 4.a.ii.A: Declare Fermi-Dirac integral sympy function
Fermi_Dirac_integral = outC.nrpyFermiDiracintegrals

# Step 4.a.ii.B: Function to compute energy moment
def energy_moment_N(N,eta_e):
    eps_N = sp.sympify(8)*M_PI/hc3 * T**(N+1) * Fermi_Dirac_integral(N,eta_e)
    return eps_N

# Step 4.a.ii.C: Compute energy moments
eps_4_plus_etae  = energy_moment_N(4,+eta_e)
eps_4_minus_etae = energy_moment_N(4,-eta_e)

<a id='beta_blocking_factors'></a>

### Step 4.a.iii: Blocking factors \[Back to [Top](#toc)\]
$$\label{beta_blocking_factors}$$

Now we compute the blocking factors:

$$
\begin{align}
\langle1-f_{\nue}\rangle_{\rm ec} &= \left\{1 + \exp\left[\eta_{\nue}-\frac{F_{5}(\eta_{\rm e})}{F_{4}(\eta_{\rm e})}\right]\right\}^{-1}\; ,\\
\langle1-f_{\nue}\rangle_{\rm pc} &= \left\{1 + \exp\left[\eta_{\anue}-\frac{F_{5}(-\eta_{\rm e})}{F_{4}(-\eta_{\rm e})}\right]\right\}^{-1}\; .
\end{align}
$$

In [7]:
# Step 4.a.iii: Blocking factors
F_4_plus_etae      = Fermi_Dirac_integral(4,+eta_e)
F_5_plus_etae      = Fermi_Dirac_integral(5,+eta_e)
F_4_minus_etae     = Fermi_Dirac_integral(4,-eta_e)
F_5_minus_etae     = Fermi_Dirac_integral(5,-eta_e)
blocking_factor_ec = one/(one + sp.exp(eta_nue  - F_5_plus_etae/F_4_plus_etae ))
blocking_factor_pc = one/(one + sp.exp(eta_anue - F_5_minus_etae/F_4_minus_etae))

<a id='beta_emission_rates'></a>

### Step 4.a.iv: Free neutrino emission rates \[Back to [Top](#toc)\]
$$\label{beta_emission_rates}$$

We now compute the free neutrino emission rates for electron capture by a proton and positron capture by a neutron:

$$
\begin{align}
  \rate{\RR}{\nue}{\rm ec} &= \frac{1+3\alpha^{2}}{8}\beta\,\rhob Y_{\rm pn}\varepsilon_{4}(+\etae)\langle1-f_{\nue}\rangle_{\rm ec}\;,\\
  %%%%
  \rate{\RR}{\anue}{\rm pc} &= \frac{1+3\alpha^{2}}{8}\beta\,\rhob Y_{\rm np}\varepsilon_{4}(-\etae)\langle1-f_{\anue}\rangle_{\rm pc}\;,
\end{align}
$$

In [8]:
# Step 4.a.iv: Free neutrino emission rates
rho_b       = sp.symbols("rho_b_cgs",real=True)
beta        = sp.symbols(namespace+"beta",real=True)
R_beta_nue  = sp.Rational(1,8)*(one + sp.sympify(3)*alpha**2)*beta*rho_b*Y_pn*eps_4_plus_etae*blocking_factor_ec
R_beta_anue = sp.Rational(1,8)*(one + sp.sympify(3)*alpha**2)*beta*rho_b*Y_np*eps_4_minus_etae*blocking_factor_pc

<a id='pair_annihilation'></a>

## Step 4.b: Electron-positron pair annihilation \[Back to [Top](#toc)\]
$$\label{pair_annihilation}$$

We now consider the blocking factor for electron-positron pair annihilation, i.e.,

$$
\ae + \ee \to \nui + \anui\; ,
$$

which is given by Eq. (B9) in [Ruffert *et al.* (1996)](https://adsabs.harvard.edu/pdf/1996A%26A...311..532R),

$$
\begin{align}
  \rate{\RR}{\nue,\anue}{\ee\ae} &= \frac{\bigl(C_{1}+C_{2}\bigr)_{\nue,\anue}}{36}\beta\,\varepsilon_{3}(\etae)\varepsilon_{3}(-\etae)\langle1-f_{\nue}\rangle_{\ee\ae}\langle1-f_{\anue}\rangle_{\ee\ae}\;,\\
  %%%%%%
  \rate{\RR}{\nux,\anux}{\ee\ae} &= \frac{\bigl(C_{1}+C_{2}\bigr)_{\nux,\anux}}{9}\beta\,\varepsilon_{3}(\etae)\varepsilon_{3}(-\etae)\bigl(\langle1-f_{\nux}\rangle_{\ee\ae}\bigr)^{2}\;,
\end{align}
$$

where

$$
\begin{align}
  \bigl(C_{1}+C_{2}\bigr)_{\nue,\anue}&=\bigl(C_V-C_{A}\bigr)^{2} + \bigl(C_{V}+C_{A}\bigr)^{2}\;,\\
  %%%%%%%
  \bigl(C_{1}+C_{2}\bigr)_{\nux,\anux}&=\bigl(C_V-C_{A}\bigr)^{2} + \bigl(C_{V}+C_{A}-2\bigr)^{2}\;,
\end{align}
$$

with $C_{A}=\frac{1}{2}$ and $C_{V} = \frac{1}{2} + 2\sin^{2}\theta_{\rm W}\approx0.96$. The blocking factors are computed using

$$
\langle1-f_{\nui}\rangle_{\ee\ae} = \left\{1 + \exp\left[\eta_{\nui} - \frac{1}{2}\frac{F_{4}(\etae)}{F_{3}(\etae)}-\frac{1}{2}\frac{F_{4}(-\etae)}{F_{3}(-\etae)}\right]\right\}^{-1}\;,
$$

with the blocking factor for antineutrinos obtained by replacing $\nui\to\anui$.

In [9]:
# Step 4.b: Electron positron pair annihilation
# Step 4.b.i: Fermi integrals
F_3_plus_etae  = Fermi_Dirac_integral(3,+eta_e)
F_3_minus_etae = Fermi_Dirac_integral(3,-eta_e)

# Step 4.b.ii: Blocking factors
common_term = sp.Rational(1,2)*(F_4_plus_etae/F_3_plus_etae + F_4_minus_etae/F_3_minus_etae)
blocking_factor_pair_nue  = one/(one + sp.exp(eta_nue  - common_term))
blocking_factor_pair_anue = one/(one + sp.exp(eta_anue - common_term))
blocking_factor_pair_nux  = one/(one + sp.exp(eta_nux  - common_term))

# Step 4.b.iii: Energy-moments
eps_3_plus_etae  = energy_moment_N(3,+eta_e)
eps_3_minus_etae = energy_moment_N(3,-eta_e)

# Step 4.b.iv: Constants
C1pC2_nue_anue = sp.symbols(namespace+"C1pC2_nue_anue",real=True)
C1pC2_nux_anux = sp.symbols(namespace+"C1pC2_nux_anux",real=True)

# Step 4.b.v: Free emission rates
common_term     = beta*eps_3_plus_etae*eps_3_minus_etae
R_pair_nue_anue = sp.Rational(1,36)*C1pC2_nue_anue*common_term*blocking_factor_pair_nue*blocking_factor_pair_anue
R_pair_nux_anux = sp.Rational(1,9) *C1pC2_nux_anux*common_term*blocking_factor_pair_nux**2

<a id='plasmon_decay'></a>

## Step 4.c: Transverse plasmon decay \[Back to [Top](#toc)\]
$$\label{plasmon_decay}$$

Next we compute the free neutrino emission rates for plasmon decay:

$$
\begin{align}
  \rate{\RR}{\nue,\anue}{\gamma} = \frac{\pi^{3}}{3\alpha_{\rm fs}}C_{V}^{2}\beta\,\frac{T^{8}}{(hc)^{6}}\gamma^{6}e^{-\gamma}(1+\gamma)\langle1-f_{\nue}\rangle_{\gamma}\langle1-f_{\anue}\rangle_{\gamma}\;,
  \label{eq:plasmon_emission_rate_nue_anue}\\
  %%%%%%%
  \rate{\RR}{\nux,\anux}{\gamma} = \frac{\pi^{3}}{3\alpha_{\rm fs}}\bigl(C_{V}-1\bigr)^{2}\beta\,\frac{T^{8}}{(hc)^{6}}\gamma^{6}e^{-\gamma}(1+\gamma)\bigl(\langle1-f_{\nux}\rangle_{\gamma}\bigr)^{2}\;,
  \label{eq:plasmon_emission_rate_nux_anux}
\end{align}
$$

where $\alpha_{\rm fs}=1/137.036$ is the fine-structure constant, $\gamma=\gamma_{0}\sqrt{\frac{1}{3}\bigl(\pi^{2}+3\etae^{2}\bigr)}$, with $\gamma_{0}=\hslash\Omega_{0}/\me c^{2} = 5.565\times10^{-2}$ related to the plasma frequency. The blocking factors are given by

$$
  \langle1-f_{\nui}\rangle_{\gamma} = \left\{1 + \exp\left[\eta_{\nui} - \left(1 + \frac{1}{2}\frac{\gamma^{2}}{1+\gamma}\right)\right]\right\}^{-1}\;,
$$

with the blocking factor for antineutrinos again obtained by replacing $\nui\to\anui$.

In [10]:
# Step 4.c: Transverse plasmon decay
# Step 4.c.i: Common terms
gamma = gamma_0*sp.sqrt(sp.Rational(1,3)*(M_PI**2 + sp.sympify(3)*eta_e**2))
common_term = sp.Rational(1,3) * M_PI**3 * beta * T**8 * gamma**6 * sp.exp(-gamma) * (one+gamma) / ( alpha_fs * hc3**2 )

# Step 4.c.ii: Blocking factors
gamma_term = one + sp.Rational(1,2)*gamma**2/(one+gamma)
blocking_factor_plasmon_nue  = one/(one + sp.exp(eta_nue  - gamma_term))
blocking_factor_plasmon_anue = one/(one + sp.exp(eta_anue - gamma_term))
blocking_factor_plasmon_nux  = one/(one + sp.exp(eta_nux  - gamma_term))

# Step 4.c.iii: Free emission rates
C_V = sp.symbols(namespace+"C_V",real=True)
R_plasmon_nue_anue = common_term * C_V**2 * blocking_factor_plasmon_nue * blocking_factor_plasmon_anue
R_plasmon_nux_anux = common_term * (C_V-1)**2 * blocking_factor_plasmon_nux**2

<a id='bremsstrahlung'></a>

## Step 4.d: Nucleon-nucleon Bremsstrahlung \[Back to [Top](#toc)\]
$$\label{bremsstrahlung}$$

Finally, we compute the free neutrino emission rates for nucleon-nucleon Bremsstrahlung:

$$
\rate{\RR}{\nui,\anui}{\rm Bremss} = C_{1}\zeta\left(X_{\rm n}^{2} + X_{\rm p}^{2} + \frac{28}{3}X_{\rm n}X_{\rm p}\right)\rhob T^{4.5}\;,
$$

where $C_{1}=2.9988\times10^{7}$, $X_{\rm n}$ and $X_{\rm p}$ are the mass fractions of the neutron and the proton, respectively, and $\zeta=0.5$ is an adhoc parameter that controls the overall amplitude of the emission rate.

In [11]:
# Step 4.d: Nucleon-nucleon Bremsstrahlung
X_n, X_p = sp.symbols("X_n X_p",real=True)
R_Brems_nui_anui = Brems_C1 * Brems_zeta * ( X_n**2 + X_p**2 + sp.Rational(28,3)*X_n*X_p ) * rho_b * T**4.5

<a id='free_cooling_rates'></a>

## Step 4.e: Free neutrino cooling rates \[Back to [Top](#toc)\]
$$\label{free_cooling_rates}$$

The corresponding neutrino cooling rates are computed using

$$
\begin{align}
  \rate{\QQ}{\nue}{\rm ec} &= \rate{\RR}{\nue}{\rm ec} \frac{\varepsilon_{5}(\etae)}{\varepsilon_{4}(\etae)}\;,\\
  %%%%%%
  \rate{\QQ}{\anue}{\rm ec} &= \rate{\RR}{\anue}{\rm pc} \frac{\varepsilon_{5}(-\etae)}{\varepsilon_{4}(-\etae)}\;,\\
  %%%%%%
  \rate{\QQ}{\nui,\anui}{\ee\ae} &= \rate{\RR}{\nui,\anui}{\ee\ae} \frac{1}{2}\frac{\varepsilon_{4}(\etae)\varepsilon_{3}(-\etae) + \varepsilon_{3}(\etae)\varepsilon_{4}(-\etae)}{\varepsilon_{3}(\etae)\varepsilon_{3}(-\etae)}\;,\\
  %%%%%%
  \rate{\QQ}{\nui,\anui}{\gamma} &= \rate{\RR}{\nui,\anui}{\gamma} \frac{1}{2}T\left(2+\frac{\gamma^{2}}{1+\gamma}\right)\;,\\
  %%%%%%
  \rate{\QQ}{\nui,\anui}{\rm Bremss} &= \rate{\RR}{\nui,\anui}{\rm Bremss}T\frac{C_{2}}{C_{1}}\;,
\end{align}
$$

where $C_{2} = 6.5428\times10^{7}$.

In [12]:
# Step 4.e: Free neutrino cooling rates
# Step 4.e.i: Energy-moments
eps_5_plus_etae  = energy_moment_N(5,+eta_e)
eps_5_minus_etae = energy_moment_N(5,-eta_e)

# Step 4.e.ii: Electron capture
Q_beta_nue = R_beta_nue * eps_5_plus_etae/eps_4_plus_etae

# Step 4.e.iii: Positron capture
Q_beta_anue = R_beta_anue * eps_5_minus_etae/eps_4_minus_etae

# Step 4.e.iv: Electron-positron pair annihilation
common_term = sp.Rational(1,2)*( eps_4_plus_etae*eps_3_minus_etae + eps_3_plus_etae*eps_4_minus_etae )/( eps_3_plus_etae*eps_3_minus_etae )
Q_pair_nue_anue = R_pair_nue_anue * common_term
Q_pair_nux_anux = R_pair_nux_anux * common_term

# Step 4.e.v: Transverse plasmon decay
common_term = sp.Rational(1,2)*T*(sp.sympify(2)+gamma**2/(one+gamma))
Q_plasmon_nue_anue = R_plasmon_nue_anue * common_term
Q_plasmon_nux_anux = R_plasmon_nux_anux * common_term

# Step 4.e.vi: Nucleon-nucleon Bremsstrahlung
Q_Brems_nui_anui = R_Brems_nui_anui * T * Brems_C2 / Brems_C1

<a id='total_rates'></a>

## Step 4.f: Total emission and cooling rates for free neutrinos \[Back to [Top](#toc)\]
$$\label{total_rates}$$

Finally, we compute the total emission and cooling rates for free neutrinos:

$$
\begin{align}
  \rate{\RR}{\nue}{\rm total} &= \rate{\RR}{\nue,\anue}{\ee\ae}
                              + \rate{\RR}{\nue,\anue}{\gamma}
                              + \rate{\RR}{\nue,\anue}{\rm Bremss}
                              + \rate{\RR}{\nue}{\rm ec}\;,\\
%%%%%%
  \rate{\RR}{\anue}{\rm total} &= \rate{\RR}{\nue,\anue}{\ee\ae}
                               + \rate{\RR}{\nue,\anue}{\gamma}
                               + \rate{\RR}{\nue,\anue}{\rm Bremss}
                               + \rate{\RR}{\anue}{\rm pc}\;,\\
%%%%%%
  \rate{\RR}{\nux}{\rm total} &= \rate{\RR}{\nux,\anux}{\ee\ae}
                              + \rate{\RR}{\nux,\anux}{\gamma}
                              + \rate{\RR}{\nux,\anux}{\rm Bremss}\;,\\
%%%%%%
  \rate{\QQ}{\nue}{\rm total} &= \rate{\QQ}{\nue,\anue}{\ee\ae}
                              + \rate{\QQ}{\nue,\anue}{\gamma}
                              + \rate{\QQ}{\nue,\anue}{\rm Bremss}
                              + \rate{\QQ}{\nue}{\rm ec}\;,\\
%%%%%%
  \rate{\QQ}{\anue}{\rm total} &= \rate{\QQ}{\nue,\anue}{\ee\ae}
                               + \rate{\QQ}{\nue,\anue}{\gamma}
                               + \rate{\QQ}{\nue,\anue}{\rm Bremss}
                               + \rate{\QQ}{\anue}{\rm pc}\;,\\
%%%%%%
  \rate{\QQ}{\nux}{\rm total} &= \rate{\QQ}{\nux,\anux}{\ee\ae}
                              + \rate{\QQ}{\nux,\anux}{\gamma}
                              + \rate{\QQ}{\nux,\anux}{\rm Bremss}\;,
\end{align}
$$

In [13]:
# Step 5: Total emission and cooling rates for free neutrinos

# Step 5.a: Electron neutrinos
R_free_total_nue = R_pair_nue_anue + R_plasmon_nue_anue + R_Brems_nui_anui + R_beta_nue
Q_free_total_nue = Q_pair_nue_anue + Q_plasmon_nue_anue + Q_Brems_nui_anui + Q_beta_nue

# Step 5.b: Electron antineutrinos
R_free_total_anue = R_pair_nue_anue + R_plasmon_nue_anue + R_Brems_nui_anui + R_beta_anue
Q_free_total_anue = Q_pair_nue_anue + Q_plasmon_nue_anue + Q_Brems_nui_anui + Q_beta_anue

# Step 5.c: Heavy lepton neutrinos or antineutrinos (single species)
R_free_total_nux  = R_pair_nux_anux + Q_plasmon_nux_anux + Q_Brems_nui_anui
Q_free_total_nux  = Q_pair_nux_anux + Q_plasmon_nux_anux + Q_Brems_nui_anui

<a id='c_function_generation'></a>

## Step 4.g: C function generation \[Back to [Top](#toc)\]
$$\label{c_function_generation}$$

Now we generate the C code for the above expressions.

In [14]:
def Cfunc_Leakage_compute_free_rates():
    desc = """(c) Leo Werneck
Compute free neutrino emission and cooling rates following
Ruffert et al. (1996)
https://adsabs.harvard.edu/pdf/1996A%26A...311..532R"""
    includes = ["Basic_defines.hh","Leakage.hh","WVU_EOS_Tabulated_headers.hh"]
    prefunc = ""
    c_type = "extern \"C\"\nvoid"
    name   = "Leakage_compute_free_rates"
    params = """const REAL rho_b,
                const REAL Y_e,
                const REAL T,
                const REAL tau_nue,
                const REAL tau_anue,
                REAL *restrict R_free_total_nue ,
                REAL *restrict R_free_total_anue,
                REAL *restrict R_free_total_nux ,
                REAL *restrict Q_free_total_nue ,
                REAL *restrict Q_free_total_anue,
                REAL *restrict Q_free_total_nux"""
    body = """
  // Step 1: Get chemical potentials and mass
  //         fractions using the EOS
  REAL mu_e, mu_n, mu_p, muhat, X_n, X_p;
  WVU_EOS_mue_mup_mun_muhat_Xn_and_Xp_from_rho_Ye_T(rho_b, Y_e, T,
                                                    &mu_e, &mu_p, &mu_n, &muhat, &X_p, &X_n);
                                                    
  // Step 2: Compute rho_b in cgs units
  const REAL rho_b_cgs = rho_b * Leakage::units_geom_to_cgs_density;

  // Step 3: Compute Y_{pn} and Y_{np}
  // Step 3.a: Compute Y_{pn} (See discussion below Eq. A8 in https://arxiv.org/pdf/1306.4953.pdf)
  REAL Y_pn;
  if( rho_b_cgs > 2e12 ) {
    // Step 3.a.i: Use Eqs. (A13) and (A14) in https://adsabs.harvard.edu/pdf/1996A%26A...311..532R
    Y_pn = (2*Y_e - 1)/(1-exp(muhat/T));
  }
  else {
    Y_pn = 1-Y_e;
  }
  // Step 3.a.ii: Make sure Y_{pn} is nonzero
  Y_pn = MAX(Y_pn,0.0);
  
  // Step 3.b: Compute Y_{np} (Eq. A13 in https://adsabs.harvard.edu/pdf/1996A%26A...311..532R)
  const REAL Y_np = exp(muhat/T) * Y_pn;

  // Step 4: The code below is generated by NRPy+
"""
    cexpr = [  R_free_total_nue ,  R_free_total_anue ,  R_free_total_nux ,  Q_free_total_nue ,  Q_free_total_anue ,  Q_free_total_nux]
    cvars = ["*R_free_total_nue","*R_free_total_anue","*R_free_total_nux","*Q_free_total_nue","*Q_free_total_anue","*Q_free_total_nux"]
    outCparams = "outCverbose=False,includebraces=False,preindent=1"
    body += outC.outputC(cexpr,cvars,"returnstring",params=outCparams).replace(namespace,"Leakage::").replace("double","REAL")
    loopopts = ""
    outC.outCfunction(os.path.join(Ccodesdir,name+".cc"),
                      includes=includes,desc=desc,prefunc=prefunc,c_type=c_type,name=name,
                      params=params,body=body,loopopts=loopopts,enableCparameters=False)

    global function_prototypes
    function_prototypes.append(c_type+" "+name+"("+params+");\n")
    function_prototypes = outC.superfast_uniq(function_prototypes)

<a id='writing_leakage_hh'></a>

# Step 5: Writing the C header file `Leakage.hh` \[Back to [Top](#toc)\]
$$\label{writing_leakage_hh}$$

We now write the `Leakage.hh` header file, which contains parameters and function prototypes for out Leakage scheme.

In [15]:
# Step 4.h: Creating the C header file Leakage.hh
def Cheader_Leakage_hh():

    # Step 4.h.i: First add all parameters which we set by hand
    Leakage_hh_string = """#ifndef LEAKAGE_HH_
#define LEAKAGE_HH_

namespace Leakage {
  // \"Primary\" parameters
"""
    for param in par.glb_Cparams_list:
        if param.module == thismodule:
            ctype = param.type
            cname = param.parname.replace(namespace,"")
            value = param.defaultval
            Leakage_hh_string += "  constexpr "+ctype+" "+cname+" = "+value+";\n"

    # Step 4.h.ii: Now add derived parameters
    C_V_impl       = C_A + sp.sympify(2)*sinthw2
    beta           = sigma_0 * c_light * N_A / m_e_c2**2
    C1pC2_nue_anue = (C_V-C_A)**2 + (C_V+C_A)**2
    C1pC2_nux_anux = (C_V-C_A)**2 + (C_V+C_A-2)**2
    param_names  = ["C_V","beta","C1pC2_nue_anue","C1pC2_nux_anux"]
    param_values = [ C_V_impl , beta , C1pC2_nue_anue , C1pC2_nux_anux ]
    Leakage_hh_string += "  // \"Derived\" parameters\n"
    for i in range(len(param_names)):
        value = param_values[i]
        cname = "constexpr REAL "+param_names[i]
        Ccode = outC.outputC(value,cname,"returnstring",params="outCverbose=false,includebraces=false").replace(namespace,"")
        Leakage_hh_string += "  "+Ccode
    Leakage_hh_string += "}\n\n// Function prototypes\n"

    # Step 4.h.iii: Finally, add the function prototypes
    global function_prototypes
    for function_prototype in function_prototypes:
        Leakage_hh_string += function_prototype+"\n"

    Leakage_hh_string += "#endif // LEAKAGE_HH_"

    # Step 4.h.iv: Write the file
    filename = os.path.join(Ccodesdir,"Leakage.hh")
    with open(filename,"w") as file:
        file.write(Leakage_hh_string)

    print("Wrote file",filename)

<a id='c_code_generation'></a>

# Step 6: C code generation \[Back to [Top](#toc)\]
$$\label{c_code_generation}$$

The function below can be used to generate all the C code implemented in this tutorial notebook.

In [16]:
def generate_NRPyLeakage_Ccode():
    Cfunc_Leakage_Fermi_Dirac_Integrals()
    Cfunc_Leakage_compute_free_rates()
    Cheader_Leakage_hh()

In [17]:
generate_NRPyLeakage_Ccode()

Output C function Leakage_Fermi_Dirac_integrals() to file standalone/NRPyLeakage/Leakage_Fermi_Dirac_integrals.cc
Output C function Leakage_compute_free_rates() to file standalone/NRPyLeakage/Leakage_compute_free_rates.cc
Wrote file standalone/NRPyLeakage/Leakage.hh
